# 03 - Legal-RAG 检索可视化 (Hybrid + 可选 Graph)

本 Notebook 用于：

1. 使用 `HybridRetriever` 对真实合同法问题进行检索；
2. 将检索结果转为表格，查看 `rank / score / 条号 / 章节 / 文本片段`；
3. 用 matplotlib 画出 top-k 的分数柱状图，直观查看检索质量；
4. 如果 `RagPipeline` 和 `law_graph` 已启用，则可选查看“Graph-augmented RAG” 的命中条文。

## 0. 切换到仓库根目录

假设本 Notebook 位于 `Legal-RAG/notebooks/03_retrieval_visualization.ipynb`。
下面这格将工作目录切换到仓库根目录，方便导入 `legalrag` 模块。

In [ ]:
import os
from pathlib import Path

NB_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
ROOT = NB_DIR.parent  # 假设 notebooks/ 在仓库根目录下
os.chdir(ROOT)
print("Notebook dir:", NB_DIR)
print("Repo root   :", ROOT)
print("Current cwd :", Path.cwd())

## 1. 导入依赖 & 初始化配置

这里使用 `AppConfig.load()` 自动加载默认配置并补全路径，然后：

- 初始化 `HybridRetriever`（FAISS + BM25）；
- 尝试初始化 `LawGraphStore` + `QueryRouter` + `RagPipeline`（如果失败则优雅降级，仅用 Hybrid）。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from legalrag.config import AppConfig
from legalrag.retrieval.hybrid_retriever import HybridRetriever
from legalrag.retrieval.graph_store import LawGraphStore
from legalrag.routing.router import QueryRouter
from legalrag.pipeline.rag_pipeline import RagPipeline
from legalrag.models import RetrievalHit

# 关闭 matplotlib 的交互模式（可选）
%matplotlib inline

# 加载配置：会自动设置 data/index 等绝对路径，并创建目录
cfg = AppConfig.load()

print("Processed file:", cfg.retrieval.processed_file)
        
# 初始化 HybridRetriever
retriever = HybridRetriever(cfg)

# 尝试初始化 graph + router + RAG pipeline（失败则降级）
graph_store = None
router = None
pipeline = None

try:
    # law_graph.jsonl 路径由 PathsConfig 提供
    graph_path = Path(cfg.paths.law_graph_jsonl)
    if graph_path.exists():
        graph_store = LawGraphStore(str(graph_path))
        print("Loaded law graph from:", graph_path)
    else:
        print("No law_graph.jsonl found at:", graph_path)

    router = QueryRouter()
    pipeline = RagPipeline(cfg)
    print("RagPipeline initialized.")
except Exception as e:
    print("[WARN] Failed to init graph/pipeline, fallback to retrieval-only mode:", e)
    graph_store = None
    router = None
    pipeline = None

## 2. 辅助函数：将 RetrievalHit 转为 DataFrame

便于查看 `rank / score / 条号 / 章节 / 段落摘要`，也方便后续画图。

In [ ]:
from typing import List

def hits_to_df(hits: List[RetrievalHit]) -> pd.DataFrame:
    rows = []
    for h in hits:
        c = h.chunk
        rows.append({
            "rank": h.rank,
            "score": h.score,
            "article_no": getattr(c, "article_no", None),
            "chapter": getattr(c, "chapter", None),
            "section": getattr(c, "section", None),
            "source": getattr(h, "source", "retriever"),
            "preview": (c.text or "")[:120].replace("\n", " ") if getattr(c, "text", None) else ""
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values("rank")
    return df

# 简单测试（如果你已经有索引）
q_test = "合同约定的违约金为合同金额的 40%，是否合理？"
hits_test = retriever.search(q_test, top_k=5)
df_test = hits_to_df(hits_test)
df_test

## 3. 可视化单个问题的 top-k score 柱状图

这里：

- 使用 HybridRetriever 检索；
- 用 DataFrame 展示 top-k 条文；
- 用 matplotlib 画出 `rank vs. score` 柱状图。

In [ ]:
def visualize_single_query(question: str, top_k: int = 8):
    print("问题：", question)
    print("-" * 80)
    hits = retriever.search(question, top_k=top_k)

    df = hits_to_df(hits)
    display(df)

    if df.empty:
        print("No hits returned.")
        return

    # 画柱状图
    plt.figure(figsize=(8, 4))
    plt.bar(df["rank"].astype(str), df["score"])
    plt.xlabel("Rank")
    plt.ylabel("Score")
    plt.title("HybridRetriever scores for: " + question[:20] + ("..." if len(question) > 20 else ""))
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.show()

# 示例问题
question_1 = "合同约定的违约金为合同金额的 40%，是否合理？"
visualize_single_query(question_1, top_k=8)

## 4. 对多组问题进行检索并比较 top-1 命中条文

在这一节中，我们对多个典型合同法问题进行检索：

- 看看每个问题的 top-1 命中条文是哪一条；
- 便于在评估时快速对比“是否命中正确章节/条号”。

In [ ]:
questions = [
    "合同约定的违约金为合同金额的 40%，是否合理？",
    "一方严重违约时，对方能否解除合同？",
    "什么是不可抗力？在合同中如何处理？",
    "当事人能否约定定金和违约金同时适用？"
]

summary_rows = []

        
for q in questions:
    hits = retriever.search(q, top_k=5)
    if not hits:
        summary_rows.append({
            "question": q,
            "top1_article_no": None,
            "top1_score": None,
            "top1_preview": "(no hit)"
        })
        continue
    h0 = hits[0]
    c0 = h0.chunk
    summary_rows.append({
        "question": q,
        "top1_article_no": getattr(c0, "article_no", None),
        "top1_score": h0.score,
        "top1_preview": (c0.text or "")[:80].replace("\n", " ") if getattr(c0, "text", None) else ""
    })

df_summary = pd.DataFrame(summary_rows)
df_summary

## 5. （可选）使用 RagPipeline + routing + graph 查看检索结果

如果你的代码中已经实现了：

- `RagPipeline` 中的 graph-augmented 检索；
        - `QueryRouter` 负责将部分问题路由到 `GRAPH_AUGMENTED` 模式；
        那么你可以使用以下函数，查看 RAG 层面的命中条文（而不只是底层 HybridRetriever）。

> 如果 `pipeline` 初始化失败（例如本地未配置 LLM），本节会自动跳过。

In [ ]:
def inspect_with_pipeline(question: str, top_k: int = 8):
    if pipeline is None:
        print("[WARN] RagPipeline 未初始化，仅支持 HybridRetriever，可回到上面的 visualize_single_query。")
        return

    print("问题：", question)
    print("=" * 80)
    ans = pipeline.answer(question, top_k=top_k)
    print("【回答】\n")
    print(ans.answer)
    print("\n【命中条文】\n")

    df = hits_to_df(ans.hits)
    display(df)

    if df.empty:
        print("No hits from pipeline.")
        return

    plt.figure(figsize=(8, 4))
    plt.bar(df["rank"].astype(str), df["score"])
    plt.xlabel("Rank")
    plt.ylabel("Score")
    plt.title("Pipeline hits (RAG / graph-augmented) for: " + question[:20] + ("..." if len(question) > 20 else ""))
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.show()

# 示例调用（如果 pipeline 初始化成功的话）
inspect_with_pipeline("什么是不可抗力？在合同中如何处理？", top_k=8)

## 6. 小结

- 这一 Notebook 展示了如何对 Legal-RAG 的检索部分进行**可视化和分析**；
- 你可以用它来：
  - 做 Hit@K 以外的“人工审阅式”评估；
  - 在面试中现场演示“检索结果 + 分数分布 + 条文内容”；
  - 比较不同检索参数（top_k、BM25 / dense 权重等）对结果排序的影响。